In [ ]:
# under19
# 대괄호, 소괄호 제외외

import pandas as pd
import time
import re
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import glob

# 🟡 파일 경로 & 출력 폴더
input_file = "kakao_작가 연재 수, 원작 여부 병합 전 최종_under19.csv"
output_dir = "output_chunks"
os.makedirs(output_dir, exist_ok=True)

# 🟡 Selenium 옵션
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 🟡 크롬 드라이버 자동 관리
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 🟡 데이터 불러오기 (CSV)
df = pd.read_csv(input_file)
title_ids = df['title_id'].dropna().astype(int).tolist()
chunk_size = 100

for i in range(0, len(title_ids), chunk_size):
    chunk_ids = title_ids[i:i+chunk_size]
    records = []

    for title_id in chunk_ids:
        url = f"https://page.kakao.com/content/{title_id}"
        driver.get(url)
        time.sleep(2)

        try:
            # 현재 작품 제목
            title_elem = driver.find_element(By.CSS_SELECTOR, "span.font-large3-bold")
            current_title = title_elem.text.strip()

            # 작가의 다른 작품 목록 찾기
            try:
                header_div = driver.find_element(By.XPATH, "//div[text()='이 작가의 다른 작품']")
                ancestor_div = header_div.find_element(By.XPATH, "./ancestor::div[contains(@class, 'w-full overflow-hidden')]")
                parent_of_list = ancestor_div.find_element(By.XPATH, "./following-sibling::div//ul")
                li_list = parent_of_list.find_elements(By.TAG_NAME, "li")
            except Exception:
                li_list = []

            normalized_title_set = set()

            for li in li_list:
                try:
                    info_div = li.find_element(By.CSS_SELECTOR, "div[aria-hidden='true'].w-full.space-y-4pxr.pb-4pxr.pr-8pxr.pt-8pxr.h-76pxr")
                    media_elem = info_div.find_element(By.CSS_SELECTOR, "span.break-all.align-middle")
                    media = media_elem.text.strip()
                    if media == "웹소설":
                        continue

                    title_elem = info_div.find_element(By.CSS_SELECTOR, "div.font-small1.line-clamp-2.break-all.text-el-60")
                    raw_title = title_elem.text.strip()
                    clean_title = re.sub(r'(\[.*?\]|\(.*?\))', '', raw_title).strip()
                    normalized_title_set.add(clean_title)
                except Exception:
                    continue

            # 현재 작품도 포함
            normalized_title_set.add(re.sub(r'(\[.*?\]|\(.*?\))', '', current_title).strip())

            records.append({
                "title_id": title_id,
                "작가_작품수": len(normalized_title_set)
            })

            print(f"[{title_id}] / 작가_작품수: {len(normalized_title_set)}")

        except Exception as e:
            print(f"[{title_id}] Error: {e}")
            records.append({
                "title_id": title_id,
                "작가_작품수": 0
            })

    # ✅ 100개 단위로 저장 (두 열만!)
    chunk_num = (i // chunk_size) + 1
    output_file = os.path.join(output_dir, f"작가_작품수_{chunk_num:04d}.csv")
    pd.DataFrame(records, columns=["title_id", "작가_작품수"]).to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"✅ {output_file} 저장 완료")

driver.quit()

# ✅ 모든 100개 단위 csv를 하나로 병합
chunk_files = sorted(glob.glob(os.path.join(output_dir, "작가_작품수_*.csv")))
dfs = []
for file in chunk_files:
    df_chunk = pd.read_csv(file, encoding="utf-8-sig")
    dfs.append(df_chunk)

if dfs:
    df_merged = pd.concat(dfs, ignore_index=True)
    merged_file = os.path.join(output_dir, "작가_작품수_전체.csv")
    df_merged.to_csv(merged_file, index=False, encoding="utf-8-sig")
    print(f"🎉 최종 통합 파일 저장: {merged_file}")
else:
    print("병합할 파일이 없습니다.")


[50866481] / 작가_작품수: 1
[56976992] / 작가_작품수: 7
[56271898] / 작가_작품수: 3
[55566760] / 작가_작품수: 4
[57260192] / 작가_작품수: 5
[58800646] / 작가_작품수: 3
[54189843] / 작가_작품수: 6
[50242834] / 작가_작품수: 3
[53190884] / 작가_작품수: 4
[55021766] / 작가_작품수: 4
[59999668] / 작가_작품수: 7
[54688834] / 작가_작품수: 2
[53297664] / 작가_작품수: 9
[60626628] / 작가_작품수: 3
[58410245] / 작가_작품수: 4
[56657309] / 작가_작품수: 3
[55872683] / 작가_작품수: 6
[56556599] / 작가_작품수: 1
[50289296] / 작가_작품수: 2
[51526841] / 작가_작품수: 2
[62346531] / 작가_작품수: 8
[52226628] / 작가_작품수: 2
[62189532] / 작가_작품수: 4
[55795261] / 작가_작품수: 7
[54526433] / 작가_작품수: 10
[62536814] / 작가_작품수: 2
[53666782] / 작가_작품수: 2
[54439672] / 작가_작품수: 4
[60194921] / 작가_작품수: 3
[64253799] / 작가_작품수: 1
[52353163] / 작가_작품수: 2
[56712146] / 작가_작품수: 1
[51185123] / 작가_작품수: 2
[46384316] / 작가_작품수: 11
[66404489] / 작가_작품수: 11
[66143747] / 작가_작품수: 11
[52792529] / 작가_작품수: 6
[58004658] / 작가_작품수: 4
[59624189] / 작가_작품수: 5
[58649535] / 작가_작품수: 6
[57519433] / 작가_작품수: 1
[65937228] / 작가_작품수: 2
[60762554] / 작가_작품수: 3
[637271

In [5]:
# under19
# 대괄호만 제외

import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import glob

# ✅ 대괄호 안의 텍스트를 제거하는 함수 (소괄호는 유지)
def remove_bracket_text(text):
    while '[' in text and ']' in text:
        start = text.find('[')
        end = text.find(']', start)
        if end == -1:
            break
        text = text[:start] + text[end+1:]
    return text.strip()

# 🟡 파일 경로 & 출력 폴더
input_file = "kakao_작가 작품 수, 원작 여부 병합 전 최종_under19.csv"
output_dir = "output_chunks"
os.makedirs(output_dir, exist_ok=True)

# 🟡 Selenium 옵션
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 🟡 크롬 드라이버 자동 관리
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 🟡 데이터 불러오기 (CSV)
df = pd.read_csv(input_file)
title_ids = df['title_id'].dropna().astype(int).tolist()
chunk_size = 100

for i in range(0, len(title_ids), chunk_size):
    chunk_ids = title_ids[i:i+chunk_size]
    records = []

    for title_id in chunk_ids:
        url = f"https://page.kakao.com/content/{title_id}"
        driver.get(url)
        time.sleep(2)

        try:
            # 현재 작품 제목
            title_elem = driver.find_element(By.CSS_SELECTOR, "span.font-large3-bold")
            current_title = title_elem.text.strip()

            try:
                header_div = driver.find_element(By.XPATH, "//div[text()='이 작가의 다른 작품']")
                ancestor_div = header_div.find_element(By.XPATH, "./ancestor::div[contains(@class, 'w-full overflow-hidden')]")
                parent_of_list = ancestor_div.find_element(By.XPATH, "./following-sibling::div//ul")
                li_list = parent_of_list.find_elements(By.TAG_NAME, "li")
            except Exception:
                li_list = []

            normalized_title_set = set()

            for li in li_list:
                try:
                    info_div = li.find_element(By.CSS_SELECTOR, "div[aria-hidden='true'].w-full.space-y-4pxr.pb-4pxr.pr-8pxr.pt-8pxr.h-76pxr")
                    media_elem = info_div.find_element(By.CSS_SELECTOR, "span.break-all.align-middle")
                    media = media_elem.text.strip()
                    if media == "웹소설":
                        continue

                    title_elem = info_div.find_element(By.CSS_SELECTOR, "div.font-small1.line-clamp-2.break-all.text-el-60")
                    raw_title = title_elem.text.strip()
                    clean_title = remove_bracket_text(raw_title)
                    normalized_title_set.add(clean_title)
                except Exception:
                    continue

            # ✅ 현재 작품도 포함
            clean_current_title = remove_bracket_text(current_title)
            normalized_title_set.add(clean_current_title)

            records.append({
                "title_id": title_id,
                "작가_작품수": len(normalized_title_set)
            })

            print(f"[{title_id}] / 작가_작품수: {len(normalized_title_set)}")

        except Exception as e:
            print(f"[{title_id}] Error: {e}")

            # ✅ header_div 자체를 못 찾았더라도, 현재 작품은 1로 포함!
            try:
                title_elem = driver.find_element(By.CSS_SELECTOR, "span.font-large3-bold")
                current_title = title_elem.text.strip()
                clean_current_title = remove_bracket_text(current_title)
                normalized_title_set = set([clean_current_title])
                작품수 = len(normalized_title_set)
            except Exception:
                작품수 = 0  # 정말 예외적으로 현재 작품조차 못 찾으면 0

            records.append({
                "title_id": title_id,
                "작가_작품수": 작품수
            })

    # ✅ 100개 단위로 저장 (두 열만!)
    chunk_num = (i // chunk_size) + 1
    output_file = os.path.join(output_dir, f"작가_작품수_{chunk_num:04d}.csv")
    pd.DataFrame(records, columns=["title_id", "작가_작품수"]).to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"✅ {output_file} 저장 완료")

driver.quit()

# ✅ 모든 100개 단위 csv를 하나로 병합
chunk_files = sorted(glob.glob(os.path.join(output_dir, "작가_작품수_*.csv")))
dfs = []
for file in chunk_files:
    df_chunk = pd.read_csv(file, encoding="utf-8-sig")
    dfs.append(df_chunk)

if dfs:
    df_merged = pd.concat(dfs, ignore_index=True)
    merged_file = os.path.join(output_dir, "작가_작품수_전체.csv")
    df_merged.to_csv(merged_file, index=False, encoding="utf-8-sig")
    print(f"🎉 최종 통합 파일 저장: {merged_file}")
else:
    print("병합할 파일이 없습니다.")


[50866481] / 작가_작품수: 1
[56976992] / 작가_작품수: 7
[56271898] / 작가_작품수: 3
[55566760] / 작가_작품수: 4
[57260192] / 작가_작품수: 5
[58800646] / 작가_작품수: 3
[54189843] / 작가_작품수: 6
[50242834] / 작가_작품수: 4
[53190884] / 작가_작품수: 4
[55021766] / 작가_작품수: 4
[59999668] / 작가_작품수: 7
[54688834] / 작가_작품수: 2
[53297664] / 작가_작품수: 9
[60626628] / 작가_작품수: 3
[58410245] / 작가_작품수: 4
[56657309] / 작가_작품수: 3
[55872683] / 작가_작품수: 6
[56556599] / 작가_작품수: 1
[50289296] / 작가_작품수: 2
[51526841] / 작가_작품수: 2
[62346531] / 작가_작품수: 8
[52226628] / 작가_작품수: 2
[62189532] / 작가_작품수: 4
[55795261] / 작가_작품수: 7
[54526433] / 작가_작품수: 10
[62536814] / 작가_작품수: 2
[53666782] / 작가_작품수: 2
[54439672] / 작가_작품수: 4
[60194921] / 작가_작품수: 3
[64253799] / 작가_작품수: 1
[52353163] / 작가_작품수: 2
[56712146] / 작가_작품수: 1
[51185123] / 작가_작품수: 2
[46384316] / 작가_작품수: 11
[66404489] / 작가_작품수: 11
[66143747] / 작가_작품수: 11
[52792529] / 작가_작품수: 6
[58004658] / 작가_작품수: 4
[59624189] / 작가_작품수: 5
[58649535] / 작가_작품수: 6
[57519433] / 작가_작품수: 1
[65937228] / 작가_작품수: 2
[60762554] / 작가_작품수: 3
[637271